In [1]:
import cv2
import time
import numpy as np

global kernel
kernel = np.ones((5,5),np.uint8)

global kernel2
kernel2 = np.ones((2,2),np.uint8)

global ALTURA_MIN
global LARGURA_MIN
global N_PIXELS_MIN

ALTURA_MIN = 1
LARGURA_MIN = 1
N_PIXELS_MIN = 20

In [5]:
class Component:
    def __init__(self, label, n_pixels):
        self.label = label
        self.n_pixels = n_pixels

def rotula (entrada, componentes, largura_min, altura_min, n_pixels_min):
    label = 0.1
    #Inicia Lista de componentes
    componentes = []
    
    for altura in range(entrada.shape[0]):
        for largura in range(entrada.shape[1]):
            #Se o pixel for branco
            if entrada[altura][largura] == 255:
                #Comece a inundação
                componente = inunda(label, entrada, altura, largura, 1)                
                #Se o objeto inundado tiver mais pixels do que o numero minimo definido
                if componente != 0:
                    #Adicione ao lista de componentes e incremente a label
                    componentes.append(componente)
                    label = label+0.1
    #printando todos os componentes encontrados
    print 'Numero de Componentes: ' + len(componentes)
    return len(componentes)


def inunda(label, entrada, x, y, ind):
    n_pixels_cont = 1
    entrada[x][y] = label
    
    #Visite seu quatro vizinhos e verifique se fazem parte do objeto
    if entrada[x-1][y] == 255:
        #Se fizerem, prossiga com a inundação recursiva nessa direção
        n_pixels_cont = inunda(label, entrada, x-1, y, 0) + 1
    if entrada[x][y-1] == 255:
        n_pixels_cont  = inunda(label, entrada, x, y-1, 0) + 1
    if entrada[x+1][y] == 255:
        n_pixels_cont = inunda(label, entrada, x+1, y, 0) + 1
    if entrada[x][y+1] == 255:
        n_pixels_cont  = inunda(label, entrada, x, y+1, 0) + 1
    
    #A variavel ind verifica se estamos na primeira chamada do objeto, assim, se a inundação foi finalizada
    if ind == 1:
        #Se for, verifique se o numero de pixels do objeto é maior que o minimo definido
        if n_pixels_cont >= N_PIXELS_MIN:
            #Se for, crie um componente e retorne
            componente = Component(label, n_pixels_cont)
            return componente
        else:
            return 0
    #Se não for a primeira chamada, retorne a contagem de pixels do objeto para ser incrementada com as outras chamadas
    else:
        return n_pixels_cont
    

In [6]:
def trataImagem(img):
    #THRESHOLD ADAPTATIVO
    imgthresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 101, -30)
    
    for img in range(1):
        #OPENING
        imgthresh = cv2.morphologyEx(imgthresh, cv2.MORPH_OPEN, kernel)
    
    #ERODE   
    imgthresh = cv2.erode(imgthresh,kernel2,iterations = 2)
    n_componentes = rotula(imgthresh, 0, ALTURA_MIN, LARGURA_MIN, N_PIXELS_MIN)
    
    return imgthresh

In [7]:
img1 = cv2.imread("60.bmp", cv2.IMREAD_GRAYSCALE )
img2 = cv2.imread("82.bmp", cv2.IMREAD_GRAYSCALE )
img3 = cv2.imread("114.bmp", cv2.IMREAD_GRAYSCALE )
img4 = cv2.imread("150.bmp", cv2.IMREAD_GRAYSCALE )
img5 = cv2.imread("205.bmp", cv2.IMREAD_GRAYSCALE )

img1thresh = trataImagem(img1)
img2thresh = trataImagem(img2)
img3thresh = trataImagem(img3)
img4thresh = trataImagem(img4)
img5thresh = trataImagem(img5)
    
cv2.imwrite('im1.bmp', img1thresh)
cv2.imwrite('im2.bmp', img2thresh)
cv2.imwrite('im3.bmp', img3thresh)
cv2.imwrite('im4.bmp', img4thresh)
cv2.imwrite('im5.bmp', img5thresh)


TypeError: cannot concatenate 'str' and 'int' objects